In [12]:
import preprocessing as preproc
import helpers
from tqdm import tqdm
import time
import text_representation as text_repr
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Embedding, Activation, Flatten, GlobalMaxPooling1D, LSTM
from keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
# def fit_tokenizer(train_data, num_words):
#     tokenizer = Tokenizer(num_words=num_words, filters='', lower=False, split=' ',
#                       char_level=False, oov_token=None, document_count=0)
#     tokenizer.fit_on_texts(train_data)
#     return tokenizer

In [3]:
# def get_embedding_matrix(tokenizer, word_embeddings, num_words):

#     embedding_matrix = np.zeros((num_words, 200))
#     for word, i in tokenizer.word_index.items():
#         if i >= num_words:
#             continue
#         if word in word_embeddings:
#             embedding_vector = word_embeddings[word]
#         else:
#             embedding_vector = np.zeros(200)
#         embedding_matrix[i] = embedding_vector
        
#     return embedding_matrix

In [4]:
start_time = time.time()

full_dataset = False
glove_pretrained = True

train_data, y, test_data = helpers.get_processed_data(full_dataset=full_dataset)

num_words = text_repr.define_num_words(train_data, full_dataset)

word_embeddings = dict()
if glove_pretrained:
    word_embeddings = text_repr.loadGloveModel("../data/twitter-datasets/glove.twitter.27B.200d.txt")

36443: num of words , 30000: will be used
Loading Glove Word Embeddings


In [5]:
tokenizer = text_repr.fit_tokenizer(train_data, num_words)

In [6]:
embedding_matrix = text_repr.get_embedding_matrix(tokenizer, word_embeddings, num_words)

In [7]:
embedding_matrix.shape

(30000, 200)

In [9]:
tweet_max_length = text_repr.define_tweet_max_len(train_data, full_dataset)

74: max sentence length found, 80: will be used


In [10]:
tokenizer = text_repr.fit_tokenizer(train_data, num_words)

train_data_seq = tokenizer.texts_to_sequences(train_data)
test_data_seq = tokenizer.texts_to_sequences(test_data)


# train_data_seq = text_repr.create_sequences(train_data, num_words)
# test_data_seq = text_repr.create_sequences(test_data, num_words)

train_data_seq = pad_sequences(train_data_seq, maxlen=tweet_max_length)
test_data_seq = pad_sequences(test_data_seq, maxlen=tweet_max_length)

In [ ]:
# keras.layers.recurrent.LSTM(output_dim, init='glorot_uniform'

In [13]:
lstm_out = tweet_max_length

In [ ]:
 dropout=0.2, recurrent_dropout=0.2

In [48]:
model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=200, weights=[embedding_matrix], 
                    input_length=tweet_max_length, trainable=True))
model.add(LSTM(units=32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(units=32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(LSTM(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))


In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 80, 200)           6000000   
_________________________________________________________________
lstm_32 (LSTM)               (None, 80, 32)            29824     
_________________________________________________________________
lstm_33 (LSTM)               (None, 80, 32)            8320      
_________________________________________________________________
lstm_34 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 66        
Total params: 6,046,530
Trainable params: 6,046,530
Non-trainable params: 0
_________________________________________________________________


In [53]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(train_data_seq, y, test_size=0.2, random_state=42)

In [55]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=32, verbose=1)

Train on 160000 samples, validate on 40000 samples
Epoch 1/5
  4992/160000 [..............................] - ETA: 18:50 - loss: 0.5680 - acc: 0.6865

KeyboardInterrupt: 

In [ ]:
# model_cnn_02 = Sequential()
# e = Embedding(num_words, 200, weights=[embedding_matrix], input_length=tweet_max_length, trainable=True)
# # e = Embedding(num_words, 200, input_length=tweet_max_length)
# model_cnn_02.add(e)
# model_cnn_02.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
# model_cnn_02.add(GlobalMaxPooling1D())
# model_cnn_02.add(Dense(256, activation='relu'))
# model_cnn_02.add(Dense(1, activation='sigmoid'))
# model_cnn_02.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])